##Imports

In [1]:
import re
import pandas as pd
import json
import numpy as np
import seaborn as sns
import nltk
from bertopic import BERTopic

/home/rodrigo98rm/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-25 22:23:57.496130: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-25 22:23:57.496148: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Extração de Dados

In [2]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

# Pré-Processamento

In [3]:
tweets = df_tweets_candidatos('./datasets/dataset.json')


# Adicionar nomes dos candidatos no dataframe
for index, row in tweets.iterrows():
    candidato = ''
    if row.author_id == 2670726740:
      candidato = 'lula'
    elif row.author_id == 128372940:
      candidato = 'bolsonaro'
    elif row.author_id == 33374761:
      candidato = 'ciro'
    else:
      candidato = 'n/d'
    tweets.at[index, 'candidato'] = candidato

print(f'Número de tweets: {len(tweets)}\n')
print(tweets.head())

Número de tweets: 300

    author_id      conversation_id                created_at  \
0  2670726740  1560319888643719168 2022-08-18 17:37:05+00:00   
1  2670726740  1560299315905515520 2022-08-18 16:15:20+00:00   
2  2670726740  1560285152282034176 2022-08-18 15:19:03+00:00   
3  2670726740  1560266077329694720 2022-08-18 14:03:15+00:00   
4  2670726740  1560248955753095168 2022-08-18 12:55:13+00:00   

                                                text                   id  \
0  As pessoas pensam que o sucesso dos nossos gov...  1560319888643719168   
1  Já em Belo Horizonte para o primeiro comício d...  1560299315905515520   
2  Ontem Lula recebeu a visita de representantes ...  1560285152282034176   
3  RT @verdadenarede: Damares condenada! No final...  1560266077329694720   
4                      Nove. https://t.co/hfMesT3LHq  1560248955753095168   

  candidato  
0      lula  
1      lula  
2      lula  
3      lula  
4      lula  


In [4]:
nltk.download('stopwords')
pt_stop = set(nltk.corpus.stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rodrigo98rm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):

        # removing urls
        document = re.sub(r'http\S+', '', document)

        # removing mentions
        document = re.sub(r'\B@\w+', '', document)

        # remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # converting to lowercase
        document = document.lower()

        # lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in pt_stop]
        tokens = [word for word in tokens if len(word) > 3]

        return ' '.join(tokens)

# Modelagem de Tópicos

In [6]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/rodrigo98rm/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/rodrigo98rm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# create model 
model = BERTopic(language="multilingual", verbose=True, min_topic_size=3, top_n_words=10, calculate_probabilities=True)

# convert to list 
docs = tweets.text.to_list()

# Pre-processamento
for i, doc in enumerate(docs):
    docs[i] = preprocess_text(doc)

tweets_topic, probabilities = model.fit_transform(docs)

Batches: 100%|██████████| 10/10 [00:00<00:00, 30.54it/s]
2022-08-25 22:24:03,007 - BERTopic - Transformed documents to Embeddings
2022-08-25 22:24:07,383 - BERTopic - Reduced dimensionality
2022-08-25 22:24:07,411 - BERTopic - Clustered reduced embeddings


Adicionando o tópico de cada tweet no Dataframe

In [16]:
topic_names = model.generate_topic_labels(nr_words=4)

# Array indicando o tópico de cada tweet (numérico -> índice do array "topic_names" + 1)
# print(tweets_topic)
# Array com todos os tópicos nomeados
# print(topic_names)

# print(docs)


for index, tweet_topic in enumerate(tweets_topic):
    tweets.at[index, 'topico'] = topic_names[tweet_topic+1]
    tweets.at[index, 'processed_text'] = docs[index]

# DEBUG
# display(tweets)
# tweets.to_csv('/home/rodrigo98rm/Documents/ufabc/pln/projeto-2/pln-2022.2/src/output/dataframe.csv', sep='\t', encoding='utf-8')

,author_id,conversation_id,created_at,text,id,candidato,topico,processed_text
0,2670726740,1560319888643719168,2022-08-18 17:37:05+00:00,As pessoas pensam que o sucesso dos nossos gov...,1560319888643719168,lula,6_brasil_país_brasildaesperança_democracia,pessoas pensam sucesso governos bolsa família ...
1,2670726740,1560299315905515520,2022-08-18 16:15:20+00:00,Já em Belo Horizonte para o primeiro comício d...,1560299315905515520,lula,11_campanha_mudar_primeiro_início,belo horizonte primeiro comício campanha próxi...
2,2670726740,1560285152282034176,2022-08-18 15:19:03+00:00,Ontem Lula recebeu a visita de representantes ...,1560285152282034176,lula,14_cironorodaviva_pergunta_ciro_responde,ontem lula recebeu visita representantes parti...
3,2670726740,1560266077329694720,2022-08-18 14:03:15+00:00,RT @verdadenarede: Damares condenada! No final...,1560266077329694720,lula,-1_ciro_presidente_brasil_reeleição,damares condenada final noite desta quarta jus...
4,2670726740,1560248955753095168,2022-08-18 12:55:13+00:00,Nove. https://t.co/hfMesT3LHq,1560248955753095168,lula,1_luladay_noite_gettr_baixar,nove
...,...,...,...,...,...,...,...,...
295,128372940,1555565663942770688,2022-08-05 14:45:29+00:00,- Independentemente de preferências ideológica...,1555565663942770688,bolsonaro,2_esperança_melhor_brasildaesperança_brasil,independentemente preferências ideológicas soa...
296,128372940,1555537798807044096,2022-08-05 12:54:46+00:00,RT @fabiofaria: Realizamos a 1ª reunião do Gov...,1555537798807044096,bolsonaro,14_cironorodaviva_pergunta_ciro_responde,realizamos reunião federal ambiente virtual ap...
297,128372940,1555521006051155968,2022-08-05 11:48:03+00:00,- A operação representa mais de R$ 20 milhões ...,1555521008248971264,bolsonaro,21_entenda_maconha_drogas_organizado,operação representa milhões prejuízo tráfico d...
298,128372940,1555521006051155968,2022-08-05 11:48:02+00:00,- Mesmo que a Polícia esteja impedida de reali...,1555521006051155968,bolsonaro,21_entenda_maconha_drogas_organizado,polícia impedida realizar certas operações rec...


Abaixo, os tópicos identificados entre todos os tweets

In [ ]:
freq = model.get_topic_info()
print(freq)

# Análise de Sentimentos

# Similaridade de Textos

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def monta_docs_tweets(df, topico, coluna_alvo, candidatos):
    lista_tweets = []
    #df_topico = df[df['topico'] == topico]
    df_topico = df

    for cand in candidatos:
        df_cand = df_topico[df_topico['candidato'] == cand]
        lista_tweets_cand = df_cand[coluna_alvo].to_list()
        lista_tweets.append(lista_tweets_cand)
    
    docs = np.concatenate(lista_tweets)
    return docs

def monta_docs_concat_topico(df, topico, coluna_alvo, candidatos):
    docs = []
    df_topico = df[(df['topico'] == topico)]
    for cand in candidatos:
        df_topico_cand = df_topico[df_topico['candidato'] == cand]
        texto = ' '.join(df_topico_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs


def monta_docs_concat_candidato(df, coluna_alvo, candidatos):
    docs = []
    for cand in candidatos:
        df_cand = df[df['candidato'] == cand]
        texto = ' '.join(df_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs

stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')

def normaliza_tweet(tweet):
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet, re.I|re.A)
    tweet = tweet.lower()
    tweet = tweet.strip()
    tokens = nltk.word_tokenize(tweet)
    tokens_filtrados = [token for token in tokens if token not in stop_words_ptbr]
    tweet_tokens = ' '.join(tokens_filtrados)
    return tweet_tokens

def vetoriza_docs(lista_docs):
    corpus_vetorizado = np.vectorize(lista_docs)
    return corpus_vetorizado

def gera_matriz_tfidf(corpus_vetorizado):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
    matriz_tfidf = tf.fit_transform(corpus_vetorizado)
    return matriz_tfidf

def similaridade_pares(matriz_tfidf):
    dict_docs_sim = cosine_similarity(matriz_tfidf)
    df_sim = pd.DataFrame(dict_docs_sim)
    return df_sim

def renomear_colunas(df, candidatos):
    df.columns = candidatos
    return df

def calcula_similaridade_topico(df, topico, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_topico(df, topico, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas(df_sim, candidatos)

    return df_sim_cands

def calcula_similaridade_candidatos(df, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_candidato(df, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas(df_sim, candidatos)

    return df_sim

def calcula_similaridade_topicos(df, topicos, coluna_alvo, candidatos):
    dfs_sim = []
    for topico in topicos:
        dfs_sim.append(calcula_similaridade_topico(df, topico, coluna_alvo, candidatos))
        
    sim_dict = dict(zip(topicos, dfs_sim))
    return sim_dict

In [ ]:
df_freq = model.get_topic_info()
topicos_mais_frequentes = df_freq['Name'].to_list()[0:4]
topicos_mais_frequentes

In [ ]:
candidatos = ["lula", "bolsonaro", "ciro"]
analise_similaridade = calcula_similaridade_topicos(tweets, topicos_mais_frequentes, "text", candidatos)
for topico in analise_similaridade.keys():
    print(f"Para tópico = {topico}")
    print(analise_similaridade[topico])
    print('----------------------------------')

In [ ]:
analise_sim_cand = calcula_similaridade_candidatos(tweets, "text", candidatos)
analise_sim_cand